# Setup

In [1]:
%reload_ext autoreload

In [2]:
%pwd

'/home/kpmurphy/github/shifty/shifty/skax'

In [3]:
%cd ..

/home/kpmurphy/github/shifty/shifty


In [4]:
%cd ..

/home/kpmurphy/github/shifty


In [129]:

from shifty.skax.logreg_flax import *
from shifty.skax.skax import *

print(loglikelihood_fn) # check that one of the symbols is defiend

<function loglikelihood_fn at 0x7f89f429b5b0>


In [6]:
# Silence WARNING:root:The use of `check_types` is deprecated and does not have any effect.
# https://github.com/tensorflow/probability/issues/1523
import logging

logger = logging.getLogger()


class CheckTypesFilter(logging.Filter):
    def filter(self, record):
        return "check_types" not in record.getMessage()


logger.addFilter(CheckTypesFilter())

In [19]:

import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=3)
import scipy.stats
import einops
from functools import partial

from functools import partial
from collections import namedtuple
import itertools
from itertools import repeat
from time import time
import chex
import typing
from typing import Any, Callable, Sequence

import jax
import jax.random as jr
import jax.numpy as jnp
from jax import vmap, grad, jit
from jax import lax, random, numpy as jnp
#import jax.debug

from flax.core import freeze, unfreeze
from flax import linen as nn
import flax

import jaxopt
import optax
import tensorflow as tf
import tensorflow_datasets as tfds

import torch
from torch.utils.data import TensorDataset, DataLoader

import distrax
from tensorflow_probability.substrates import jax as tfp
tfd = tfp.distributions
from tensorflow_probability.substrates.jax.distributions import MultivariateNormalFullCovariance as MVN

#jax.config.update("jax_enable_x64", False)


import sklearn.datasets
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LogisticRegression

In [8]:
import os 
cpu_count = os.cpu_count()
print(cpu_count)

# Run jax on multiple CPU cores
# https://github.com/google/jax/issues/5506
# https://stackoverflow.com/questions/72328521/jax-pmap-with-multi-core-cpu
import os 
#os.environ["XLA_FLAGS"] = '--xla_force_host_platform_device_count=90'

import jax
print(jax.devices())

96
[CpuDevice(id=0)]


# Logreg

In [ ]:
from shifty.skax.logreg_flax import *

print(loglikelihood_fn) # check that one of the symbols is defiend

In [10]:
def make_test_data():
    iris = sklearn.datasets.load_iris()
    X = iris["data"]
    #y = (iris["target"] == 2).astype(np.int)  # 1 if Iris-Virginica, else 0'
    y = iris["target"]
    nclasses = len(np.unique(y)) # 
    ndata, ndim = X.shape  # 150, 4
    key = jr.PRNGKey(0)
    noise = jr.normal(key, (ndata, ndim)) * 2.0
    X = X + noise # add noise to make the classes less separable
    #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    X = np.array(X)
    return X, y


print(X.shape)
print(y.shape)
print(type(X))
#print(type(y))

(150, 4)
(150,)
<class 'numpy.ndarray'>


In [33]:
X,y = make_test_data()
# We set C to a large number to turn off regularization.
# We don't fit the bias term to simplify the comparison below.
l2reg = 1e-2
log_reg = LogisticRegression(solver="lbfgs", C=1/l2reg, fit_intercept=True)
log_reg.fit(X, y)
W_mle = log_reg.coef_ # (nclasses, ndim)
b_mle = log_reg.intercept_ # (nclasses,)
true_probs = log_reg.predict_proba(X)
nclasses, ndim = W_mle.shape
ntrain = X.shape[0]



In [35]:
key = jr.PRNGKey(0)
nclasses, ndim = W_mle.shape
print([nclasses, ndim])
key = jr.PRNGKey(0)
model = LogReg(key,  nclasses, W_init=W_mle.T, b_init=b_mle, l2reg=l2reg)
probs = np.array(model.predict(X))

model = LogReg(key, nclasses)
model.fit(X, y)
probs = np.array(model.predict(X))

print('pred: ', probs[0])
print('truth: ', true_probs[0])


[3, 4]


/home/kpmurphy/mambaforge/lib/python3.10/site-packages/distrax/_src/utils/conversion.py:143: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return x.astype(jnp.float_)


pred:  [0.851 0.14  0.009]
truth:  [0.858 0.134 0.008]


In [36]:
def compare_method(optimizer, name=None, batch_size=ntrain, max_iter=500):
    key = jr.PRNGKey(0)
    model = LogReg(key, nclasses, max_iter=max_iter, l2reg=l2reg, optimizer=optimizer, batch_size=batch_size)  
    model.fit(X, y)
    probs = np.array(model.predict(X))
    print('method {:s}, max deviation from true probs {:.3f}'.format(name, np.max(true_probs - probs)))
    print('truth: ', true_probs[0])
    print('pred: ', probs[0])

In [37]:
compare_method("LBFGS", "LBFGS")

method LBFGS, max deviation from true probs 0.011
truth:  [0.858 0.134 0.008]
pred:  [0.855 0.137 0.009]


In [38]:
compare_method(optax.adam(0.01), name="adam 0.01, bs=N", batch_size=ntrain)

method adam 0.01, bs=N, max deviation from true probs 0.163
truth:  [0.858 0.134 0.008]
pred:  [0.801 0.169 0.03 ]


In [39]:
compare_method(optax.adam(0.01), name="adam 0.01, bs=32", batch_size=32)

method adam 0.01, bs=32, max deviation from true probs 0.235
truth:  [0.858 0.134 0.008]
pred:  [0.771 0.185 0.044]


In [40]:
compare_method("polyak", name="polyak, bs=32", batch_size=32)

method polyak, bs=32, max deviation from true probs 0.645
truth:  [0.858 0.134 0.008]
pred:  [0.735 0.256 0.009]


In [41]:
compare_method("armijo", name="armijo, bs=32", batch_size=32)

method armijo, bs=32, max deviation from true probs 0.251
truth:  [0.858 0.134 0.008]
pred:  [0.757 0.199 0.044]


# Logreg pipeline

In [43]:

import sklearn
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import LogisticRegression

def fit_classifier_sklearn(key, X, Y):
    classifier = Pipeline([
            ('standardscaler', StandardScaler()),
            ('poly', PolynomialFeatures(degree=2)), 
            ('logreg', LogisticRegression(random_state=0, max_iter=100, C=1e5))])
    classifier.fit(np.array(X), np.array(Y))
    return classifier
    
key = jr.PRNGKey(0)
clf = fit_classifier_sklearn(key, X, y)

true_probs = clf.predict_proba(X)
print(true_probs[0])
print(np.sum(true_probs[0]))

[8.617e-01 1.380e-01 2.624e-04]
1.0000000000000002


In [46]:

def fit_classifier_skax(key, X, Y):
    ndim = X.shape[1]
    nclasses  = len(np.unique(Y))
    classifier = Pipeline([
            ('standardscaler', StandardScaler()),
            ('poly', PolynomialFeatures(degree=2)), 
            ('logreg', LogReg(key, nclasses, max_iter=500, l2reg=1e-5))])
    classifier.fit(np.array(X), np.array(Y))
    return classifier
    
key = jr.PRNGKey(0)
model = fit_classifier_skax(key, X, y)
probs = np.array(model.predict(X))
print('max deviation from true probs {:.3f}'.format(np.max(true_probs - probs)))
print('truth: ', true_probs[0])
print('pred: ', probs[0])

/home/kpmurphy/mambaforge/lib/python3.10/site-packages/distrax/_src/utils/conversion.py:143: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return x.astype(jnp.float_)


max deviation from true probs 0.006
truth:  [8.617e-01 1.380e-01 2.624e-04]
pred:  [8.606e-01 1.391e-01 2.525e-04]


# Skax logreg

In [147]:
X,y = make_test_data()
l2reg = 1e-2
log_reg = LogisticRegression(solver="lbfgs", C=1/l2reg, fit_intercept=True)
log_reg.fit(X, y)
true_probs = log_reg.predict_proba(X)
ninputs = X.shape[1]
ntrain = X.shape[0]
nclasses = len(np.unique(y))


In [148]:
def compare_skax_method(optimizer, name=None, batch_size=ntrain, max_iter=500):
    key = jr.PRNGKey(0)
    network = LogRegNetwork(nclasses = nclasses)
    model = NeuralNetClassifier(network, key, nclasses, l2reg=l2reg, optimizer = optimizer, batch_size=batch_size, max_iter=max_iter) 
    model.fit(X, y)
    probs = np.array(model.predict(X))
    print('method {:s}, max deviation from true probs {:.3f}'.format(name, np.max(true_probs - probs)))
    print('truth: ', true_probs[0])
    print('pred: ', probs[0])

In [149]:
compare_skax_method("LBFGS", "LBFGS")



fit ninputs  4


/home/kpmurphy/mambaforge/lib/python3.10/site-packages/distrax/_src/utils/conversion.py:143: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return x.astype(jnp.float_)


method LBFGS, max deviation from true probs 0.011
truth:  [0.858 0.134 0.008]
pred:  [0.855 0.137 0.009]


In [55]:

compare_skax_method("armijo", name="armijo, bs=32", batch_size=32)

/home/kpmurphy/mambaforge/lib/python3.10/site-packages/distrax/_src/utils/conversion.py:143: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return x.astype(jnp.float_)


method armijo, bs=32, max deviation from true probs 0.251
truth:  [0.858 0.134 0.008]
pred:  [0.757 0.199 0.044]


# Neural nets with no hidden layers

In [160]:
def fit_classifier_sklearn(key, X, Y):
    classifier = Pipeline([
            ('standardscaler', StandardScaler()),
            ('poly', PolynomialFeatures(degree=2)), 
            ('logreg', LogisticRegression(random_state=0, max_iter=100, C=1e5))])
    classifier.fit(np.array(X), np.array(Y))
    return classifier
    
key = jr.PRNGKey(0)
clf = fit_classifier_sklearn(key, X, y)

true_probs_pipeline = clf.predict_proba(X)
print(true_probs_pipeline[0])
print(np.sum(true_probs_pipeline[0]))

[8.617e-01 1.380e-01 2.624e-04]
1.0000000000000002


In [161]:
def fit_classifier_skax_mlp(key, X, Y):
    ndim = X.shape[1]
    nclasses  = len(np.unique(Y))
    network = MLPNetwork((nclasses,))
    model = NeuralNetClassifier(network, key, nclasses, l2reg=1e-5, max_iter=500)
    classifier = Pipeline([
            ('standardscaler', StandardScaler()),
            ('poly', PolynomialFeatures(degree=2)), 
            ('logreg', model)])
    classifier.fit(np.array(X), np.array(Y))
    return classifier
    
key = jr.PRNGKey(0)
model = fit_classifier_skax_mlp(key, X, y)
probs = np.array(model.predict(X))
print('max deviation from true probs {:.3f}'.format(np.max(true_probs_pipeline - probs)))
print('truth: ', true_probs_pipeline[0])
print('pred: ', probs[0])

/home/kpmurphy/mambaforge/lib/python3.10/site-packages/distrax/_src/utils/conversion.py:143: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return x.astype(jnp.float_)


max deviation from true probs 0.004
truth:  [8.617e-01 1.380e-01 2.624e-04]
pred:  [8.622e-01 1.375e-01 2.553e-04]


In [156]:
compare_mlp_method("LBFGS", "LBFGS")

fit ninputs  4
method LBFGS, max deviation from true probs 0.011
truth:  [0.858 0.134 0.008]
pred:  [0.855 0.137 0.009]


In [157]:

compare_skax_method("armijo", name="armijo, bs=32", batch_size=32)

fit ninputs  4
method armijo, bs=32, max deviation from true probs 0.251
truth:  [0.858 0.134 0.008]
pred:  [0.757 0.199 0.044]


# Neural nets with hidden layers

In [162]:
def fit_classifier_skax_mlp(key, X, Y):
    ndim = X.shape[1]
    nclasses  = len(np.unique(Y))
    network = MLPNetwork((5, nclasses,))
    model = NeuralNetClassifier(network, key, nclasses, l2reg=1e-5, max_iter=500)
    classifier = Pipeline([
            ('standardscaler', StandardScaler()),
            ('poly', PolynomialFeatures(degree=2)), 
            ('logreg', model)])
    classifier.fit(np.array(X), np.array(Y))
    return classifier
    
key = jr.PRNGKey(0)
model = fit_classifier_skax_mlp(key, X, y)
probs = np.array(model.predict(X))
print('max deviation from true probs {:.3f}'.format(np.max(true_probs_pipeline - probs)))
print('truth: ', true_probs[0])
print('pred: ', probs[0])

/home/kpmurphy/mambaforge/lib/python3.10/site-packages/distrax/_src/utils/conversion.py:143: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in astype is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return x.astype(jnp.float_)


max deviation from true probs 0.934
truth:  [8.617e-01 1.380e-01 2.624e-04]
pred:  [1.00e+00 8.77e-26 0.00e+00]
